# 01 Load FRED File

In [ ]:
SELECT current_database() AS DATABASE_NAME, current_schema() AS SCHEMA_NAME

In [ ]:
# Import python packages
import logging
import pandas as pd

logger = logging.getLogger("fred_logger")

# Get the target database and schema using the results from the SQL cell above
# This won't be needed when we can pass variables to Notebooks!
current_context_df = cells.db_schema_table.to_pandas()
database_name = current_context_df.iloc[0,0]
schema_name = current_context_df.iloc[0,1]

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()
#session.use_schema(f"{database_name}.{schema_name}")

logger.info("01_load_files start")

In [ ]:
SELECT '@INTEGRATIONS.S3_FRED_STAGE/T10Y2Y.xlsx' AS STAGE_FILE_PATH, 'Daily' AS WORKSHEET_NAME, 'FRED_10Y_2Y' AS TARGET_TABLE

In [ ]:
import os
from openpyxl import load_workbook

def load_excel_worksheet_to_table_local(session, stage_file_path, worksheet_name, target_table):
  local_directory = "./"
  file_name = os.path.basename(stage_file_path)

  # First copy file from stage to local storage
  get_status = session.file.get(stage_file_path, local_directory)

  with open(f"{local_directory}{file_name}", 'rb') as f:
    workbook = load_workbook(f)
    sheet = workbook[worksheet_name]
    data = sheet.values

    # Get the first line in file as a header line
    columns = next(data)[0:]
    # Create a DataFrame based on the second and subsequent lines of data
    df = pd.DataFrame(data, columns=columns)
 
    df2 = session.create_dataframe(df)
    df2.write.mode("overwrite").save_as_table(target_table)
 
  return True

In [ ]:
# Process each file from the sql_get_spreadsheets cell above
files_to_load = cells.sql_get_spreadsheets.to_pandas()
for index, excel_file in files_to_load.iterrows():
    logger.info(f"Processing Excel file {excel_file['STAGE_FILE_PATH']}")
    load_excel_worksheet_to_table_local(session, excel_file['STAGE_FILE_PATH'], excel_file['WORKSHEET_NAME'], excel_file['TARGET_TABLE'])

logger.info("01_load_files end")